# Trending Youtube Video Analysis 
#### by Sam Logsdon, Maryam Bokhari, and Jeong Rae Park

# Loading Data from the Database 
This will initialize the database if your directories are setup according to the readme. 

In [ ]:
import db
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os.path

DB_PATH = 'db.sqlite'
DATA_PATH = 'data'

if not (os.path.exists(DB_PATH)):
    import db_loader
    skipped = db_loader.run(DATA_PATH, DB_PATH)
    

db = db.DB(DB_PATH)
base_df = db.fetch_videos_as_df() # make a copy of this using base_df.copy()
categories = db.fetch_categories()

---

# Question 1

## Shared Videos Between Countries

We are interested in viewing the proportion of shared trending videos between countries. The data for this analysis was calculated using a self-join of the videos table in sqlite on the basis of video_id and trending_date. The country was selected from each side of the join as well, and rows where both countries were the same were excluded. 

In [ ]:
import os.path

SQL = '''
SELECT v1.country c1, v2.country c2, count(*) videos
             FROM video v1
                      JOIN video v2
                           ON (v2.video_id = v1.video_id) AND (v2.trending_date = v1.trending_date)
             WHERE c1 != c2
GROUP BY c1, c2
'''
rows = db.conn.execute(SQL).fetchall()
df = pd.DataFrame.from_records(data = rows, columns=['c1', 'c2', 'videos'])
df.head()

In [ ]:
total_videos_df = base_df.groupby(level=[0]).agg(total_videos=('views', 'count'))
total_videos_df.join(df.set_index('c1'), rsuffix='c1').join(df.set_index('c2'), lsuffix='_c1',rsuffix='_c2')

In [ ]:

total_videos_df['key'] = 0
vddf = total_videos_df.reset_index().merge(total_videos_df.reset_index(), on='key').drop('key', axis=1).set_index(['country_x', 'country_y'])
vddf.index = vddf.index.rename(['c1', 'c2'])
df = df.set_index(['c1', 'c2']).join(vddf)

In [ ]:

df['total_videos'] = df.total_videos_x + df.total_videos_y
df['shared_percent'] = df.videos / df.total_videos 


## Pivoted Dataframe
We use the pandas pivot function to reshape our data. Note that the data is mirrored across the line of missing(nan) values. This is because the self-join query returned data for (c1,c2) and (c2,c1) pairs e.g. (US,RU) and (RU,US). We will clean this up in the next step. 

In [ ]:
pv = df.reset_index().pivot(index='c1', columns='c2', values='shared_percent')
pv.rename_axis(None).style.set_table_styles([{'selector': '.index_name', 'props': [('font-size', '0')]}])

## Generated heatmap

In [ ]:
sns.set(style="ticks", font_scale=1.2)
%matplotlib inline
plt.figure(figsize=(12,8)) 


mask = np.zeros_like(pv, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

ax = sns.heatmap(pv, cmap = sns.color_palette("YlGnBu", 10), 
                 annot=True, annot_kws = {'size': 'large'}, fmt='.2%',
                 mask = mask,
                 cbar = False
                )
# fix for mpl bug that cuts off top/bottom of seaborn viz
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values

ax.xaxis.get_major_ticks()[-1].set_visible(False)
ax.yaxis.get_major_ticks()[0].set_visible(False)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
ax.set_xlabel('')
ax.set_ylabel('');

## Average proportion of shared videos

In [ ]:
df.groupby('c1').aggregate(
    average_shared=('shared_percent', 'mean')).sort_values(by='average_shared',ascending=False)

---

# Question 2

## Average time for video to go from viewable to trending

In [ ]:
m = base_df.copy()

The preprocessing for this question required us to drop duplicate Video Id's such that we keep the first instance after we sort the data.

Sample Video ID (Before Pre-processing): 

| Video ID   |   Trending Date   | Publish Date |
|----------|-------------|------|
| 1 |  2018-09-01. | 2018-09-01 |
| 1 |  2018-09-02.  |   2018-09-01|
| 1 |  2018-09-03.  |   2018-09-01 |


Sample Video ID (After Pre-processing): 

| Video ID   |   Trending Date   | Publish Date |
|----------|-------------|------|
| 1 |  2018-09-01. | 2018-09-01 |


In [ ]:
#Pre processing data- Dropping Duplicate Values and handling null values 
m = m.sort_values(by=['country', 'video_id', 'trending_date'])
group = m.groupby(level=[1])
m = group.first()

The trending age wascalculated by subtracting the trending time with the publish time. 

In [ ]:
m['trending_date']=m['trending_date'].dt.normalize()
m['publish_time']=m['publish_time'].dt.normalize()
m['time_to_trending']= m['trending_date'].sub(m['publish_time'])
m['time_to_trending'].describe()
m['time_to_trending'] = m['time_to_trending'].dt.days

In [ ]:
#plotting the first most frequent timings
import matplotlib.pyplot as plt
import scipy.stats

counts = m.groupby('time_to_trending').agg(count=('likes', 'count')).sort_values(by='time_to_trending')
#y=m['time_to_trending'].sort_values().value_counts()#[:10]
counts[:15].plot(kind='bar', title="Frequency of Time taken to trend videos", figsize=(15,8))


It's a shame we don't have a higher resolution for trending_date to get a proper sense of the distribution. From this view, it looks like it could be normally distributed, but the shape could change significantly if we zoomed in.

In [ ]:
#Data distribution
maxi=m['publish_time'].max()
mini=m['publish_time'].min()
print(mini, '-', maxi)
print(m['trending_date'].count() )

In [ ]:
#taking a year with the most number of published videos 
avg=m.groupby(by=[m.publish_time.dt.year]).count()
avg

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
grouped = m.groupby('trending_date').agg(average_time_to_trend=('time_to_trending', 'mean'), 
                                         error=('time_to_trending', 'std'), 
                                         mad=('time_to_trending', 'mad'))
grouped.plot(kind='line', y='average_time_to_trend',figsize=(15,10))
plt.show()

Youtube has clearly made some changes to their trending algorithm to prefer more recent content. Before the change, videos from as early as 2006 could trend on the basis of just a few hundred views, causing a huge variance in age. 

---

# Question 3

# What type of video category is most likely to become trending.

We are interested in viewing list of categories which is most likely to become trending.


In [ ]:
q3 = base_df.copy()

With this process, we can count number of vedio in each category in each countries.

In [ ]:
q3 = q3.groupby(['country', 'category_id']).agg(
    count = ('likes', 'count')
)
q3

With this code, we can sort the data by count which is number of video and give a rank based on count.
After that, we only pull out the top 5 from each group

In [ ]:
q3 = q3.sort_values(by=['country', 'count'], ascending=False)
temp_df = q3.groupby(['country']).agg(rank = ('count', 'cumcount'))
q3 = pd.merge(q3, temp_df, left_index=True, right_index=True)
q3 = q3.join(categories.set_index('id'), on=['category_id'])


In this code, we join two data, q3 and category. 
After that, we aggregated all number of videos in categories which are not in top 6, and renamed the category id as "99" and ranked it as "7". 

In [ ]:
#delete this block before submitting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
%matplotlib inline

bottom = q3[q3['rank']>6].groupby(level=[0]).agg(count=('count', 'sum'))
bottom['category_id'] = 99
bottom['category'] = 'Other'
bottom['rank'] = 7
bottom = bottom.reset_index().set_index(['country', 'category_id'])
#q3 = q3[q3['rank']<=6].append(bottom, sort=True)
q3

make barplot and swarmplot.
With swarmplot, we can see that rank of categories base of number of videos in each countries.
As you see, Entertainment category is popular in most countries, but not all counties.
There are similarity and difference by countries.
With barplot, we cass see that mean of number of videos in each countries.
in 5 countries, Entertainment category is outliers which means that it is way over popular than other category there.

In [ ]:
plt.figure(figsize=(12,8)) 
#ax = sns.barplot(x=df3.index.get_level_values(1), y="count",hue=df3.index.get_level_values(1), data=df3)
sns.boxplot(x='country', y='count', data=q3.reset_index(), color='.75')
ax = sns.swarmplot(x="country", y="count", data=q3[q3['rank']<5].reset_index(), hue='category',hue_order=q3.sort_values('category').category.drop_duplicates(), size=12)


plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

# Question 4

## Popular videos at same time in all counties

We are interested in viewing the list of popular videos trended at same time in all countries. 

The preprocessing of the data for this analysis was calculated using a self-join of the videos table in sqlite on the basis of video_id and trending_date. By using SQL, we can pull out the videos which are trended at the same time in countries.

In [ ]:
from containers import YoutubeVideo

videos = []
with db.conn:
    for row in db.conn.execute("SELECT count(video_id) as c, * FROM video WHERE video_id != '#NAME?' GROUP BY trending_date, video_id ORDER BY c DESC "):
        videos.append(row)

df4 = pd.DataFrame.from_records(data=videos, columns=['c'] + list(YoutubeVideo._fields))
df4

With Dataframe, we get the videos which has the count number over 10. It means that the video which has count number over 10 are trended in all counties at same time because we have only 10 counties data. There are some same videos twice or more being trended at the same time in all counties which is duplication, but we did not delete this duplicate because our porpose is to see the list of vedios which are become trending at the same time in all counties. 

In [ ]:
df_filtered = df4.query('c>=10')
df_filtered = df_filtered.sort_values(by=['title'], ascending=False)
df_new = df_filtered[['title','channel_title','trending_date']]

# Table chart

We only pull out the some columns, title and channel_title and trending_date, and make table chart to view it.

In [ ]:
import warnings
warnings.filterwarnings('ignore') #supress font warnings

collabel=("title","channel_title","trending_date")

fig, ax = plt.subplots()

fig.patch.set_visible(False)

ax.axis('off')
ax.axis('tight')

table_df=ax.table(cellText=df_new.values, colLabels=collabel, loc='center')
table_df.auto_set_font_size(False)
table_df.set_fontsize(12)
table_df.scale(3.5,2.5)

fig.tight_layout()

plt.show()

# Predicting Categories With Machine Learning
We used an SGDClassifier because of the memory requirements of many other models

In [ ]:
df = db.fetch_videos_as_df(exclude=['tags', 'thumbnail_link'])
df = df.reset_index().groupby('video_id').first()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

X = df[['country' ,'title','description']]
target = df.category_id
X_train, X_test, target_train, target_test = train_test_split(X, target, test_size=0.4, random_state=0, stratify=target)

column_trans = ColumnTransformer(
     [('country_category', OneHotEncoder(), ['country']),
     ('description_t', TfidfVectorizer(), 'description')])
svm_model = make_pipeline(column_trans, SGDClassifier(loss="hinge", penalty="l2"))
svm_model.fit(X_train, target_train)
svm_labels = svm_model.predict(X_test)


print(classification_report(target_test, svm_labels))


In [ ]:
model = make_pipeline(TfidfVectorizer(), SGDClassifier(loss="hinge", penalty="l2"))
model.fit(X_train['description'], target_train)
labels = model.predict(X_test['description'])
print(classification_report(target_test, labels))

Using description alone seems to be about the same as both description and country